In [1]:
!pip install gym

    100% |████████████████████████████████| 1.5MB 8.7MB/s 
    100% |████████████████████████████████| 1.0MB 11.2MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/6c/3a/0e/b86dee98876bb56cdb482cc1f72201035e46d1baf69d10d028
Successfully built gym


In [0]:
import tensorflow as tf
import numpy as np
import gym

In [3]:
# Create the env
env = gym.make('FrozenLake8x8-v0')

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [0]:
# Get the action_space and observation_space
action_size = env.action_space.n
observation_size = env.observation_space.n

In [0]:
tf.reset_default_graph()

# The Q-Network

In [0]:
# Hyperparameters of the network
lr = 0.8
max_steps = 99
gamma = .99
total_episodes = 5000

# Exploration parameters
epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.3
decay_rate = 0.01

# Other paramters
interval = 50
tboard_dir = './tboard'

In [0]:
# Declare the placeholders
inputs = tf.placeholder(shape=[1, observation_size], dtype=tf.float32)
tqvals = tf.placeholder(shape=[1, action_size], dtype=tf.float32)

In [0]:
# Build the model

W1 = tf.get_variable('qweights', shape=[observation_size, action_size], 
                     initializer=tf.random_normal_initializer())

pout = tf.matmul(inputs, W1)
predict = tf.argmax(pout, axis=1)

In [0]:
# Define the loss function and optimizer
criterion = tf.reduce_sum(tf.square(tqvals - pout))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(criterion)

In [0]:
# TensorBoard Ops
def attach_summaries1(var):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    
    stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
    
attach_summaries(W1)
attach_summaries(criterion)

In [49]:
## Traning the network

init = tf.global_variables_initializer()

reward_list = []
steps_list = []

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(tboard + '/train/', sess.graph)
    
    sess.run(init)
    
    for episode in range(total_episodes):
        
        state = env.reset()
        done = False
        total_rewards = 0
        
        for step in range(max_steps):
            
            action, curr_qvals = sess.run([predict, pout], 
                                       feed_dict={inputs: np.identity(observation_size)[state:state+1]})
            action = action[0]
            
            exp_prob = np.random.uniform()
            if exp_prob < epsilon:
                action = env.action_space.sample()
                
                
            #print (action, curr_qvals)
            
            # Step with the chosen action
            new_state, rewards, done, info = env.step(action)
            
            # Get the qvalues for the new state
            next_qvals = sess.run([pout], 
                         feed_dict={inputs: np.identity(observation_size)[new_state:new_state+1]})
            
            # Get the target output for current state
            max_next_qvals = np.max(next_qvals)
            target_q = curr_qvals
            target_q[0, action] = rewards + (gamma * max_next_qvals)
            
            # Train the network using the target Q values
            _ = sess.run([train_op], feed_dict={inputs: np.identity(observation_size)[state:state+1],
                                                tqvals: target_q})
            
            
            total_rewards += rewards
            
            if done or step == max_steps:
                if step % interval == 0:
                    env.render()
                    print ('Maximum steps: {}'.format(step))
                break
                
            state = new_state
        
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
        
        reward_list.append(total_rewards)
        steps_list.append(step)
        
        if step % interval == 0:
            print ('Episode {}/{}'.format(episode, total_episodes),
                   'Reward: {}'.format(total_rewards))
        
print ('Agent Trained successfully!!')
print ('Rewards over time: {:.3f}'.format(sum(reward_list) / total_episodes))

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
Maximum steps: 50
Episode 74/5000 Reward: 0.0
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
Maximum steps: 50
Episode 226/5000 Reward: 0.0
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
Maximum steps: 50
Episode 267/5000 Reward: 0.0
  (Left)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
Maximum steps: 50
Episode 300/5000 Reward: 0.0
  (Left)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
Maximum steps: 50
Episode 456/5000 Reward: 0.0
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
Maximum steps: 50
Episode 653/5000 Reward: 0.0
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
Maximum steps: 50
Episode 715/5000 Reward: 0.0
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
Maximum steps: 50
Episode 

KeyboardInterrupt: ignored

In [0]:
# Testing Hyperparameters
total_episodes = 100
max_steps = 99
interval = 1

In [59]:
# Testing out the network

init = tf.global_variables_initializer()

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    
    merged = tf.summary.merge_all()
    test_writer = tf.summary.FileWriter(tboard_dir + '/test/', sess.graph)
    sess.run(init)
    
    for episodes in range(total_episodes):
        
        state = env.reset()
        done = False
        all_rewards = 0
        
        for step in range(max_steps):
            next_action = sess.run([predict], 
                          feed_dict={inputs: np.identity(observation_size)[state:state+1]})
            
            next_action = next_action[0][0]
            
            next_state, rewards, done, info = env.step(next_action)
            
            all_rewards += rewards
            
            if done or step == max_steps:
                if step % interval == 0:
                    env.render()
                    print ('Episode {}/{}'.format(episode, episode+1),
                           'Rewards: {}'.format(all_rewards),
                           'Maximum steps: {}'.format(step+1))

KeyboardInterrupt: ignored

data  sample_data
mkdir: cannot create directory ‘data’: File exists


In [62]:
!ls tboard/test/


events.out.tfevents.1543511596.95319df47ea2


In [0]:
cd